In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
from datetime import datetime
from itertools import tee, islice, chain
from datetime import timedelta

Вводные

Водитель может находиться в 4 возможных статусах:
    * free -- доступен для нового заказа
    * enroute -- едет на заказ
    * ontrip -- выполняет заказ
    * busy -- недоступен для нового заказа

Возможные переходы из одного состояние в другое определены как:
    * free -> [free, enroute, busy]
    * enroute -> [free, ontrip]
    * ontrip -> [free]
    * busy -> [free]

Почему переходы определяются таким образом:
1. Из состояния free можно перейти в
    * free -- если водитель ушел в офлайн и заново вышел на линию, тогда подряд будет две записи со статусом free
    * enroute -- если водитель принял заказ, то переходит в статус enroute и едет к клиенту
    * busy -- если водитель нажал кнопку "Занят" в таксометре (пошел на обед и т.д.)
2. Из состояния enroute можно перейти в 
    * free -- если клиент или водитель отменил заказ
    * ontrip -- если водитель приехал к клиенту и начал выполнять заказ
3. Из состояния ontrip можно перейти только в free (после выполнения заказа)
4. Из состояния busy можно перейти только в free

Эффективность на поездке -- это время с клиентом в машине (ontrip), деленное на сумму длительностей всех статусов, связанных с поездкой (sum(free) + enroute + ontrip), где sum(free) -- время простоя.

Время простоя -- это сумма всех статусов free, предшествующих поездке. Суммируются все статусы free, идущие подряд, а также те, которые были прерваны короткими статусами busy или enroute (короткий статус == меньше какого-то TIMEOUT'а).

Имеется набор данных со статусами водителей, по которому необходимо построить зависимость длительности поездки от эффективности.
    * driver_id -- id водителя
    * status -- один из статусов
    * dttm -- время начала статуса

Примечания:
    * Поездка считается только при наличии статуса ontrip
    * Тесты написаны для python 2

    1. Написать функцию-генератор, которая будет отдавать соседние элементы в цикле. Функция понадобится для итерирования по записям водителя и проверки соседних статусов по условиям. Не забудьте проверить, что тесты проходят без ошибок (см. test_neighbors).

In [2]:
from itertools import tee, islice, chain

def neighbors(iterable):
    prv, cur, nxt = tee(iterable, 3)
    prv = chain([None], prv)
    nxt = chain(islice(nxt, 1, None), [None])
    yield prv, cur, nxt       

In [3]:
# Check if test passes
def test_neighbors():
    test_neighbors = neighbors( range(2) )
    assert test_neighbors.next() == (None, 0, 1)

test_neighbors()

AttributeError: 'generator' object has no attribute 'next'

    2. Сгруппировать данные на уровне водителя таким образом, чтобы в одной строке находились все его записи со статусами и началом статуса списком:

Формат исходной таблицы:
<table>
<tr><td>driver_id</td><td>status</td><td>dttm</td></tr>
<tr><td>9f8f9bf3ee8f4874873288c246bd2d05</td><td>free</td><td>2018-02-04 00:19</td></tr>
<tr><td>9f8f9bf3ee8f4874873288c246bd2d05</td><td>busy</td><td>2018-02-04 01:03</td></tr>
<tr><td>8f174ffd446c456eaf3cca0915d0368d</td><td>free</td><td>2018-02-03 15:43</td></tr>
<tr><td>8f174ffd446c456eaf3cca0915d0368d</td><td>enroute</td><td>2018-02-03 17:02</td></tr>
<tr><td>...</td><td>...</td><td>...</td></tr>
</table>

Формат сгруппированной таблицы:
<table>
<tr><td>driver_id</td><td>driver_info</td></tr>
<tr><td>9f8f9bf3ee8f4874873288c246bd2d05</td><td>[("free", 2018-02-04 00:19), ("busy", 2018-02-04 01:03)]</td></tr>
<tr><td>8f174ffd446c456eaf3cca0915d0368d</td><td>[("free", 2018-02-03 15:43), ("enroute", 2018-02-03 17:02) ...]</td></tr>
</table>

In [4]:
df = pd.read_csv("dataset.csv", parse_dates=["dttm"])


In [5]:
df = df.sort_values('driver_id')

In [6]:
df.head()

,status,driver_id,dttm
2260,enroute,029e9b80b45c483fbb00c4695c2d9c09,2018-02-19 02:38:31.620339
2253,enroute,029e9b80b45c483fbb00c4695c2d9c09,2018-02-18 20:42:57.620339
2254,ontrip,029e9b80b45c483fbb00c4695c2d9c09,2018-02-18 21:50:19.620339
2255,free,029e9b80b45c483fbb00c4695c2d9c09,2018-02-18 23:06:20.620339
2256,enroute,029e9b80b45c483fbb00c4695c2d9c09,2018-02-19 00:01:00.620339


In [7]:
drivers_list = df.driver_id.unique()

In [8]:
big_list = []
for i in drivers_list:
  
    df1 = df[df['driver_id'] == i]
    
    list1 = []
    for j in df1.values:
        list1.append((j[0], j[2]))
    big_list.append([i, list1])

In [9]:
df_grouped = pd.DataFrame(big_list, columns=['driver_id', 'driver_info'])

In [10]:
df_grouped.head()

,driver_id,driver_info
0,029e9b80b45c483fbb00c4695c2d9c09,"[(enroute, 2018-02-19 02:38:31.620339), (enrou..."
1,041ddc65a16649129eb6dbee9b54d6e8,"[(free, 2018-02-19 13:37:36.620339), (free, 20..."
2,048459e02e63450d9429ceb34601929d,"[(enroute, 2018-02-19 17:00:04.620339), (ontri..."
3,0f8eccff510f4bdabbff9f40b2262517,"[(ontrip, 2018-02-21 22:21:05.620339), (free, ..."
4,14637bc848b24ac49fbadc43d207e375,"[(enroute, 2018-02-19 09:22:37.620339), (ontri..."


    3. Используя функцию neighbors, написать функцию, которая для каждой записи в списке driver_info посчитает ее длительность.

In [11]:
def calc_status_duration(driver_info):
    driver_info_updated = []
    for i, j, k in neighbors(driver_info):
         timedelta(i[1], j[1])
    return driver_info_updated

In [12]:
# Check if test passes
def test_calc_status_duration():
    sample_driver_info = [("free", datetime(2018, 4, 2, 0, 19)), 
                          ("busy", datetime(2018, 4, 2, 1, 3)),]
    sample_driver_info_updated = [('free', datetime(2018, 4, 2, 0, 19), 2640.0),
                                  ('busy', datetime(2018, 4, 2, 1, 3), None),]
    assert calc_status_duration(sample_driver_info) == sample_driver_info_updated

test_calc_status_duration()

TypeError: 'itertools.chain' object is not subscriptable

In [13]:
df_grouped["driver_info"] = df_grouped.driver_info.apply(calc_status_duration)

TypeError: 'itertools.chain' object is not subscriptable

    4. Используя функцию neighbors, написать функцию, которая сформирует из списка driver_info список поездок с информацией о длительности поездки и эффективности (duration_ontrip, efficiency).

In [14]:
TIMEOUT = 1600

def collapse_statuses(driver_info):
    # Here define conditions under which the "free" state 
    # should be attributed to the trip.
    # ... type your code here ...

SyntaxError: unexpected EOF while parsing (<ipython-input-14-746238299881>, line 6)

In [15]:
# Check if test passes
def test_collapse_statuses():
    sample_driver_info = [("free", datetime(2018, 4, 2, 0, 19), 2640.0), 
                          ("busy", datetime(2018, 4, 2, 1, 3), 1660.0),
                          ("free", datetime(2018, 4, 2, 1, 30, 40), 2050.0),
                          ("enroute", datetime(2018, 4, 2, 2, 4, 50), 70.0),
                          ("free", datetime(2018, 4, 2, 2, 6), 500.0),
                          ("enroute", datetime(2018, 4, 2, 2, 14, 20), 520.0),
                          ("ontrip", datetime(2018, 4, 2, 2, 23), 3060.0),
                          ("free", datetime(2018, 4, 2, 3, 14), None)
                         ]
    sample_driver_info_updated = [(3060.0, 3060.0 / (3060.0 + 520.0 + 500.0 + 2050.0))]
    assert collapse_statuses(sample_driver_info) == sample_driver_info_updated

test_collapse_statuses()

NameError: name 'collapse_statuses' is not defined

In [16]:
df["driver_info"] = df.driver_info.apply(collapse_statuses)

AttributeError: 'DataFrame' object has no attribute 'driver_info'

    5. Нарисовать и проинтерпретировать зависимость между длительностью поездки и эффективностью.
    Подсказка: требуется сделать обратное преобразование из таблицы со строками на уровне водителя в таблицу со строками на уровне поездки.